In [24]:
from pathlib import Path

import pickle
import geopandas as gpd
import pandas as pd

# Code to convert windthrow flags from tsbrowser into a shapefile

In [36]:
pickles = list(
    Path("/mnt/share/HR-VPP2/Calibration/Interpretation/windthrow/Flags").glob(
        "*.pickle"
    )
)
shp = gpd.read_file(
    "/mnt/share/HR-VPP2/Calibration/Interpretation/windthrow/Partitions/interp_tool_samples.shp"
)
shp

,Id_poly,EventDate,StormName,EventType,Country,Area,Perimeter,Damage_deg,Methods,Dataprovid,Source,Id,EventId,max_inscri,centroid,data_loc,confidence,comment,geometry
0,21049,2018-10-28,Gudrun,Windstorm,SE,0.032742,129.05990359193,-999.0,Semiautomatic classification,"Forest national service of Sweden, Sweden",http://skogsdataportalen.skogsstyrelsen.se/Sko...,60,SE20181028,5.167534,POINT (4557316.890953046 3892239.360468839),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,HIGH,no apparent wt,POINT (14.00079 58.07212)
1,21050,2018-10-28,Gudrun,Windstorm,SE,0.022688,79.92413581274,-999.0,Semiautomatic classification,"Forest national service of Sweden, Sweden",http://skogsdataportalen.skogsstyrelsen.se/Sko...,61,SE20181028,5.660858,POINT (4557167.1844357485 3892180.925566224),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,HIGH,no apparent wt,POINT (13.99821 58.07167)
2,21076,2018-10-28,Gudrun,Windstorm,SE,0.047611,122.41720163427,-999.0,Semiautomatic classification,"Forest national service of Sweden, Sweden",http://skogsdataportalen.skogsstyrelsen.se/Sko...,87,SE20181028,5.494770,POINT (4546628.981836695 3948956.2911087563),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,LOW,border - regrowth,POINT (13.87461 58.58656)
3,21079,2018-10-28,Gudrun,Windstorm,SE,0.041179,94.11526212484,-999.0,Semiautomatic classification,"Forest national service of Sweden, Sweden",http://skogsdataportalen.skogsstyrelsen.se/Sko...,90,SE20181028,7.503370,POINT (4408095.644956825 3942253.7970261155),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,HIGH,no apparent wt,POINT (11.49436 58.57324)
4,21089,2018-10-28,Gudrun,Windstorm,SE,0.049969,109.76795569805,-999.0,Semiautomatic classification,"Forest national service of Sweden, Sweden",http://skogsdataportalen.skogsstyrelsen.se/Sko...,100,SE20181028,8.691156,POINT (4476367.622677097 3786134.355068685),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,LOW,None,POINT (12.56487 57.15269)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,-1,2022-08-07,unknown,Windstorm,LV,1.272518,637.114020389722,-999.0,Aerial photointerpretation and field survey,Copernicus Emergency Service,https://mapping.emergency.copernicus.eu/activa...,13903,LV20220807,36.047900,POINT (5306279.731649619 3864815.208697314),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,LOW,unclear,POINT (26.30478 56.87101)
568,-1,2022-08-07,unknown,Windstorm,LV,1.650557,537.3135221064313,-999.0,Aerial photointerpretation and field survey,Copernicus Emergency Service,https://mapping.emergency.copernicus.eu/activa...,14629,LV20220807,55.637269,POINT (5299931.890126086 3912274.228989821),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,MEDIUM,border,POINT (26.38622 57.29761)
569,-1,2022-08-07,unknown,Windstorm,LV,0.311786,215.6785973711477,-999.0,Aerial photointerpretation and field survey,Copernicus Emergency Service,https://mapping.emergency.copernicus.eu/activa...,14038,LV20220807,28.024186,POINT (5280734.027895776 3898205.0432694517),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,LOW,unclear,POINT (26.02159 57.21371)
570,-1,2022-08-07,unknown,Windstorm,LV,0.190217,175.01537290421507,-999.0,Aerial photointerpretation and field survey,Copernicus Emergency Service,https://mapping.emergency.copernicus.eu/activa...,14885,LV20220807,21.575003,POINT (5373330.062098322 3811043.3205336276),//digs110/FER/HR-VPP2/Calibration/Chips/Windth...,LOW,unclear,POINT (27.15899 56.26346)


In [32]:
event_labels = {"5", "6", "7", "8", "9", "a", "c", "d"}

In [46]:
rows = []
for flag_path in pickles:
    sample_id = flag_path.stem.split("_")[-1]
    with flag_path.open("rb") as f:
        flags = pickle.load(f)
    segments = {}
    events = {}
    for time, label in flags.items():
        time_str = time.strftime("%Y%m%dT%H%M%S")
        if label in event_labels:
            events[label] = [time_str]
        else:
            if segments.get(label) is None:
                segments[label] = []
            segments[label].append(time_str)
            segments[label].sort()
    rows.append(
        {
            "Id": int(sample_id),
            "interpret1": "vij",
            "segments": segments,
            "events": events,
        }
    )

In [49]:
wt_df = pd.DataFrame(rows)
output = shp.merge(wt_df, on="Id").rename(columns={"Id": "id"})

In [50]:
# write shapefile
output.to_file("data/windthrow_flags.geojson")